### The first cell in this file should always be executed, then one can choose a particular experiment to run.

In [ ]:
#Tom Laeven (4247078) en Marnix Huibers (4144015)

from numba import jit
import numpy as np
import matplotlib.pyplot as plt
import functionsArgon3 as fA

### 1. Standard Simulation (energy checks)

In [ ]:
settings = fA.md_settings(TSI=119.8,
                          rho=np.sqrt(2),
                          NumberOfBoxesPerDimension=2,
                          NumberOfTimeSteps=2000,
                          TruncR=3000,
                          h=0.004)
# Initialise the system
r, v, Virial, U, Ekin, instance = fA.initialise(settings,seed=101)

# The Simulation
for q in range(0,settings.NumberOfTimeSteps):
    instance = fA.velocity_verlet(instance, settings)
    r[:,:,q+1] = instance.r
    v[:,:,q+1] = instance.v
    U[q+1] = instance.U
    Ekin[q+1] = instance.Ekin

# Plotting
TimeAxis = np.arange(0,settings.NumberOfTimeSteps+1)
Etot = Ekin + U
plt.title("Energy plot")
plt.xlabel("Time Steps")
plt.ylabel("Energy")
plt.xlim([0, settings.NumberOfTimeSteps])
plt.plot(TimeAxis, Ekin,label="Kinetic Energy")
plt.plot(TimeAxis, U,label="Potential Energy")
plt.plot(TimeAxis, Etot, label="Total Energy")
plt.legend(loc = 5,bbox_to_anchor=(1.51,0.8))
plt.show()

### 2. Heat Capacity

In [ ]:
settings = fA.md_settings(TSI=3*119.8,
                          rho=0.01,
                          NumberOfBoxesPerDimension=6,
                          NumberOfTimeSteps=100000,
                          TruncR=3,
                          h=0.004)
seed = 101

#Give the input parameters
Eq = 6000 #Equilibrium time
Ti = 2000
NumTi = int(np.floor((settings.NumberOfTimeSteps -Eq)/Ti))
#settings = fA.md_settings({"TSI":TSI, "rho":rho, "NumberOfBoxesPerDimension":NumberOfBoxesPerDimension,
#                "NumberOfTimeSteps":NumberOfTimeSteps, "TruncR":TruncR, "h":h})
# Initialise the system
r, v, Virial, U, Ekin, instance = fA.initialise(settings,seed)
## rescale
labda = np.sqrt( ((settings.N-1)*3*settings.T)/(2*Ekin[0]) )
v[:,:,0] *= labda
Ekin[0] *= labda*labda
instance.v = v[:,:,0]
# The Simulation
for q in range(0,settings.NumberOfTimeSteps):
    instance = fA.velocity_verlet(instance, settings)
    r[:,:,q+1] = instance.r
    v[:,:,q+1] = instance.v
    U[q+1] = instance.U
    Ekin[q+1] = instance.Ekin
    if q < Eq: ## rescale
        labda = np.sqrt( ((settings.N-1)*3*settings.T)/(Ekin[q+1]*2) )
        v[:,:,q+1] *= labda
        Ekin[q+1] *= labda*labda
        instance.v = v[:,:,q+1]
# calculate Cv
Cv = np.zeros(NumTi)
for i in range(0,NumTi):
    EkInt = Ekin[(Eq+Ti*i):(Eq+Ti*(i+1))]
    Cv[i]=(3*settings.N*np.mean(EkInt)*np.mean(EkInt))/(2*np.mean(EkInt)*np.mean(EkInt)-
                                                   3*settings.N*np.var(EkInt))
CvMean = np.mean(Cv)
CvStd = np.std(Cv)

print("Calculated heat capacity:", CvMean/settings.N, "±", CvStd/settings.N/np.sqrt(NumTi), "         Dulong Petit:", 3,"          Ideal Gas:", 1.5)

#plotting Ekin
TimeAxis = np.arange(0,settings.NumberOfTimeSteps+1)
plt.title("Kinetic Energy plot")
plt.xlabel("Time Steps")
plt.ylabel("Kinetic Energy")
plt.xlim([0, settings.NumberOfTimeSteps])
plt.plot(TimeAxis, Ekin)
plt.show()

### 3. Pressure

In [ ]:
settings = fA.md_settings(TSI=0.99*119.8,
                          rho=0.6,
                          NumberOfBoxesPerDimension=6,
                          NumberOfTimeSteps=30000,
                          TruncR=3,
                          h=0.001)
seed = 101

Eq = 20000 #Equilibrium time
Ti = 500
NumTi = int(np.floor((settings.NumberOfTimeSteps -Eq)/Ti))

# Initialise the system
r, v, Virial, U, Ekin, instance = fA.initialise(settings,seed)

labda = np.sqrt( ((settings.N-1)*3*settings.T)/(2*Ekin[0]) )
v[:,:,0] *= labda
Ekin[0] *= labda*labda

instance.v = v[:,:,0]

# The Simulation
for q in range(0,settings.NumberOfTimeSteps):
    instance = fA.velocity_verlet(instance, settings)
    r[:,:,q+1] = instance.r
    v[:,:,q+1] = instance.v
    U[q+1] = instance.U
    Ekin[q+1] = instance.Ekin
    Virial[q+1] = instance.Virial
    
    if q < Eq:
        labda = np.sqrt( ((settings.N-1)*3*settings.T)/(Ekin[q+1]*2) )
        v[:,:,q+1] *= labda
        Ekin[q+1] *= labda*labda
        
        instance.v = v[:,:,q+1]

P = np.zeros(NumTi)
for i in range(0,NumTi):
    VirialInt = Virial[(Eq+Ti*i):(Eq+Ti*(i+1))]
    P[i] =  (settings.rho * settings.T - (settings.rho/(3*settings.N)) * np.mean(VirialInt) - 
            2*np.pi*settings.rho* settings.rho/3 *  8/(settings.TruncR**3) )

PMean = np.mean(P)
PStd = np.std(P)

print("Calculated pressure per rho*T:", PMean/(settings.rho*settings.T), "±", PStd/np.sqrt(NumTi),
      "         Ideal gas:", 1)


TimeAxis = np.arange(0,settings.NumberOfTimeSteps+1)
Etot = Ekin + U
plt.plot(P)
plt.show()

### 5. Correlation function

In [ ]:
settings = fA.md_settings(TSI=1*119.8,
                          rho=0.8,
                          NumberOfBoxesPerDimension=6,
                          NumberOfTimeSteps=4000,
                          TruncR=10,
                          h=0.001)
seed = 101

Eq = 2000 #Equilibrium time
Ti = 1000
NumTi = int(np.floor((settings.NumberOfTimeSteps -Eq)/Ti))

# Initialise the system
r, v, Virial, U, Ekin, instance = fA.initialise(settings,seed)

labda = np.sqrt( ((settings.N-1)*3*settings.T)/(2*Ekin[0] ))
v[:,:,0] *= labda
Ekin[0] *= labda*labda
CorrHist = np.zeros((200,(settings.NumberOfTimeSteps-Eq)))

instance.v = v[:,:,0]

# The Simulation
for q in range(0,settings.NumberOfTimeSteps):
    instance = fA.velocity_verlet(instance, settings)
    r[:,:,q+1] = instance.r
    v[:,:,q+1] = instance.v
    U[q+1] = instance.U
    Ekin[q+1] = instance.Ekin
    Virial[q+1] = instance.Virial
    
    if q < Eq:
        labda = np.sqrt( ((settings.N-1)*3*settings.T)/(Ekin[q+1]*2) )
        v[:,:,q+1] *= labda
        Ekin[q+1] *= labda*labda
        
        instance.v = v[:,:,q+1]
    else:
        CorrHist[:,q-Eq] = instance.hist[:200]

HistAverage = np.mean(CorrHist,1)
Dis = np.arange(0,settings.NumberOfTimeSteps+1)
plt.title("Kinetic Energy plot")
plt.xlabel("Time Steps")
plt.ylabel("Kinetic Energy")
plt.xlim([0, settings.NumberOfTimeSteps])
plt.plot(TimeAxis, Ekin)
plt.show()


### Diffusion constant

In [ ]:
settings = fA.md_settings(TSI=1*119.8,
                          rho=0.8,
                          NumberOfBoxesPerDimension=6,
                          NumberOfTimeSteps=10000,
                          TruncR=10,
                          h=0.004)
seed = 104

Eq = 2500 #Equilibrium time
Ti = 1000
NumTi = int(np.floor((settings.NumberOfTimeSteps -Eq)/Ti))

# Initialise the system
r, v, Virial, U, Ekin, instance = fA.initialise(settings,seed)

routside = r

labda = np.sqrt( ((settings.N-1)*3*settings.T)/(2*Ekin[0] ))
v[:,:,0] *= labda
Ekin[0] *= labda*labda
CorrHist = np.zeros((200,(settings.NumberOfTimeSteps-Eq)))

instance.v = v[:,:,0]

for q in range(0,settings.NumberOfTimeSteps):
    instance = fA.velocity_verlet(instance, settings)
    routside[:,:,q+1] = routside[:,:,q] + instance.dr
    
    v[:,:,q+1] = instance.v
    Ekin[q+1] = instance.Ekin
   
    if (q < Eq)|(q%20==0):
        labda = np.sqrt( ((settings.N-1)*3*settings.T)/(Ekin[q+1]*2) )
        v[:,:,q+1] *= labda
        Ekin[q+1] *= labda*labda 
        instance.v = v[:,:,q+1]

ABT = 1000
        
Diffdist = np.mean(np.sum((routside[:,:,Eq:] - routside[:,:,Eq,np.newaxis])**2,0),0)
TimeAxis = settings.h*np.arange(ABT,settings.NumberOfTimeSteps+1-Eq)
Coeff = np.polyfit(TimeAxis,Diffdist[ABT:],1)
D = Coeff[0]/6

plt.plot(settings.h*np.arange(0,settings.NumberOfTimeSteps+1-Eq),Diffdist)
plt.plot(settings.h*np.arange(0,settings.NumberOfTimeSteps+1-Eq),settings.h*np.arange(0,settings.NumberOfTimeSteps+1-Eq)*6*D+Coeff[1])
plt.show()